# Running pcarpet

## Imports

In [1]:
import os
import sys
# import pcarpet
pcarpet_path = os.path.join(os.path.dirname(os.getcwd()), 'pcarpet')
sys.path.insert(0, pcarpet_path)
import pcarpet

## Initializing pcarpet
`pcarpet` needs 3 paths to be specified:

1. fMRI file: a 4d nifti file containing the preprocessed fMRI data
2. Mask file: a 3d nifti file, containing a binary mask that defines a single region-of-interest (e.g. cortex)
3. Output folder: for storing the outputs generated by pcarpet
   
The fMRI and Mask files need to be in the same space, with identical x-y-z dimensions. For example, they can both be in the native anatomical space of the subject, or in a template space to which the data have been resampled.

In [5]:
# Folder containing example data
example_folder = '/home/niko/MRI/pcarpet_example/macaque'

# 1. Path to preprocessed fMRI file
func = os.path.join(example_folder, 'func_preproc.nii.gz')
# 2. Path to a cortical mask
cortex_mask = os.path.join(example_folder, 'cortex_mask.nii.gz')
# 3. Path to a folder for storing the outputs
output_folder = os.path.join(example_folder, 'outputs')

To initialize pcarpet, we first create `Dataset` object, using the three paths from above

In [6]:
MyData = pcarpet.Dataset(func, cortex_mask, output_folder)


Initialized Dataset object:
	fMRI file: /home/niko/MRI/pcarpet_example/macaque/func_preproc.nii.gz
	Mask file: /home/niko/MRI/pcarpet_example/macaque/cortex_mask.nii.gz
	Output directory: /home/niko/MRI/pcarpet_example/macaque/outputs


Once the `Dataset` object is created, we have two ways of running the `pcarpet` pipeline on it:

1. Running the entire pipeline at once
2. Running the pipeline step-by-step
   
Below, we will first demonstrate the convenient first option. We will then go through the step-by-step option, which may be useful for debugging and for understanding the vairous inputs and outputs of the pipeline.

## Option 1: Running the entire pcarpet pipeline at once
We can directly call the `run_pcarpet` method of the `Dataset` object, which will execute the pipeline with default options.

In [7]:
MyData.run_pcarpet()

Reading data...
	fMRI data read: dimensions (80, 33, 80, 600)
	Mask read: dimensions (80, 33, 80)
fMRI data reshaped to voxels x time (211200, 600).
21589 voxels retained after masking.
Carpet matrix created with shape (21589, 600).
Carpet normalized to zero-mean unit-variance.
Carpet reordered.
PCA fit to carpet and results saved.
First 5 PCs correlated with carpet.
Out of these, 3 sign-flipped.
First 5 PCs correlated with fMRI data.
TR of 2.000 seconds read from fMRI header
Visual report generated and saved as First5_PCs_carpet_corr_report.


We can see what the default options were, since they are stored in a dictionary called `used_options`.

In [8]:
ops = MyData.used_options
for key, val in ops.items():
    print(f'{key}: {val}')

tSNR_thresh: 15.0
reorder_carpet: True
save_carpet: False
save_pca_scores: False
ncomp: 5
flip_sign: True
TR: auto


Any of the above default options can be overriden by explicitly passing them as arguments to the `run_pcarpet` call. For example: 

```python
MyData.run_pcarpet(tSNR_thesh=10, reorder_carpet=False)
```

The meaning of each of these option will become clear in the step-by-step guide that follows.

## Option 2: Running pcarpet step-by-step
### Step 1: Importing the data
The first function, `import_data`, uses the `nibabel` package to import the fMRI data and the Mask into python as numpy arrays, and stores them as object attributes `data` and `mask`, respectiveyl. It also stores other attributes, like the x-y-z-t dimensions of fMRI data, the nifti header and the affine matrix.

In [9]:
MyData.import_data()

Reading data...
	fMRI data read: dimensions (80, 33, 80, 600)
	Mask read: dimensions (80, 33, 80)


### Step 2: Getting the carpet
The second function, `get_carpet`, generates a 'carpet' from the fMRI data and the mask. A carpet is a 2d matrix, shaped voxels x time, which contains the normalized (z-score) BOLD-fMRI signal from within the mask.

The fMRI data is first reshaped to 2d. Voxels that are outside the mask as well as voxels below a specified temporal signal-to-noise ratio threshold `tSNR_thresh` (default = 15) are discarded. The retained voxels are tranformed into 2d and normalized through z-scoring (subtract mean and divide by standard deviation along the time dimension).

By default, the rows of the carpet matrix (voxels) are ordered according to their (decreasing) correlation with the global (mean across voxels) signal. The reordering helps to highlight widespread signal fluctuations. It can be turned off by setting the `reorder_carpet` argument to `False`.

The carpet matrix is stored as a `carpet` attribute of the `Dataset` object. Optionally, it can be written to the output folder as a `carpet.npy` file (can be large), by setting the `save_carpet` argument to `True`.

In [10]:
MyData.get_carpet(tSNR_thresh=15.0,
                  reorder_carpet=True, save_carpet=False)

fMRI data reshaped to voxels x time (211200, 600).
21589 voxels retained after masking.
Carpet matrix created with shape (21589, 600).
Carpet normalized to zero-mean unit-variance.
Carpet reordered.


### Step 3: Fit PCA to carpet
The third function, `fit_pca2carpet`, fits PCA to the `carpet` matrix and saves the principal componens (PCs), the explained variance ratios, and optionally the PCA scores (PCA-tranformed carpet). 

The PCs are stored as a `pca_comps` attribute of the `Dataset` object and are also written to the output folder as a `PCs_all.npy` file. 

The explained variance ratios per PC are stored as a `expl_var` attribute of the `Dataset` object and are also written to the output folder as a `PCs_all_expl_variance_ratio.npy` file.

If the `save_pca_scores` option is set to `True`, the PCA-transformed data will be written to the output folder as a `PCA_scores_all.npy` file (will be large).

In [11]:
MyData.fit_pca2carpet(save_pca_scores=False)

PCA fit to carpet and results saved.
